<a href="https://colab.research.google.com/github/skevin-dev/NLP-FELLOWSHIP/blob/week4/Kevin_Hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## In this notebook, following tasks will be conducted: 
1. Write a program which takes array of urls for job-posting websites in Rwanda
   * https://www.jobinrwanda.com/  (Job Adverts)
   * https://www.umucyo.gov.rw  ( Tenders)

2. Web-scrap the content and put them in a pandas data-frame.

3. Use regular expression and key-words to retrieve IT/Software related job/consultancy opportunities(Tenders).

4. Use EasyNMT to display the results in 50+ languages using wrapped state of art models.

5. Hosting of our work example in colab

## Data Extraction 

In [2]:
# import Libraries 

import requests # for requesting a link
from bs4 import BeautifulSoup # to scrap information fron a web 
import pandas as pd # for data wrangling/manipulation 


In [8]:
def DataExtraction(link):
  """ This function helps to extract data about job in rwanda given a link 
  
  Args
  ----
  link:str
      link of the page to be scrapped 

  returns
  -------
  df:dataframe
     a dataset containing all information 
  """
  content =  requests.get(link).content
  soup =  BeautifulSoup(content, "html.parser")


  # find all jobs
  list_of_jobs = soup.find_all('span',class_='field field--name-title field--type-string field--label-hidden')
  jobs_titles = [list_of_jobs[index].text for index in range(len(list_of_jobs))]

  # extract companies 
  content_ = soup.find_all('p',class_='card-text')
  companies = [content_[index].find('a')['href'].split('/')[-1] for index in range(len(content_))]

  # extract experience
  experience = [content_[index].text.split('\n')[-3].strip() for index in range(len(content_))]

  # extract published dates 
  published_dates = [content_[index].text.split('\n')[-6].strip().split(' ')[2] for index in range(len(content_))] 

  # extract deadlines 
  deadlines_list = soup.find_all('time',class_='datetime')

  deadlines = [deadlines_list[d].text for d in range(len(deadlines_list))]
  
  # job description
  descriptions = soup.find_all('div',class_='card-body p-2')
  descriptions_links = ['https://www.jobinrwanda.com'+descriptions[index].find('a')['href']for index in range(len(descriptions))]
  
  # creating empty lists 
  location= []
  position = []
  contract = []
  education = []
  description = []
  # iterate through list of description links 
  for link_ in descriptions_links:
    content_ =  requests.get(link_).content
    soup_ =  BeautifulSoup(content_, "html.parser")
    b= soup_.find('ul',class_="list-group list-group-flush").text
    c =  b.split('\n')
    # remove unnecessary white spaces 
    res = [ele for ele in c if ele.strip()]

    # retrieving information
    location.append(res[2].split(' ')[-1])
    position.append(res[-2].split(' ')[-1])
    contract.append(res[-4].split(' ')[-1])
    if res[-7].split(' ')[-2] == ':':
      education.append(res[-7].split(' ')[-1])
    else:
      education.append(" ".join(res[-7].split(' ')[-2:]))
    #description
    des = soup_.find('div',class_='clearfix text-formatted field field--name-field-job-full-description field--type-text-long field--label-hidden field__item').text
    description.append(des.split('\n'))
  # create an empty dataframe 
  df = pd.DataFrame()

  # create columns and assigning values 
  df['Job Titles'] = jobs_titles
  df['Çompanies'] = companies 
  df['Experience'] = experience
  df['Published Date']  =published_dates
  df['Deadline'] = deadlines
  df['Location'] = location
  df['Number of Position'] = position
  df['Contract Type'] = contract
  df['Education Level']=education
  df['Description content'] =description
 
  return df 

In [9]:
DataExtraction('https://www.jobinrwanda.com/jobs/all')

,Job Titles,Çompanies,Experience,Published Date,Deadline,Location,Number of Position,Contract Type,Education Level,Description content
0,Procurement Manager,access-finance-rwanda-afr,Senior (5+ years of experience),31-10-2022,10-11-2022,RW,1,Full-time,Master,[Advertisement for Recruitment of the Procurem...
1,Procurement Officer,american-dream-home-international-adhi-rwanda-ltd,Mid career (3 to 5 years of experience),04-11-2022,10-11-2022,RW,1,Full-time,Bachelor,"[Job Brief, We are looking for an astute procu..."
2,HR Coordinator,american-dream-home-international-adhi-rwanda-ltd,Senior (5+ years of experience),04-11-2022,10-11-2022,RW,1,Full-time,Bachelor,"[Job brief, ADHI Rwanda Ltd is a newly establi..."
3,Project Engineer,american-dream-home-international-adhi-rwanda-ltd,Mid career (3 to 5 years of experience),04-11-2022,10-11-2022,RW,1,Full-time,Bachelor,"[Job Brief, American Dream Home International-..."
4,Customer Service Officer,tic-tac-toe,Entry level (1 to 3 years of experience),03-11-2022,11-11-2022,RW,2,Full-time,High school,"[Job Description & Responsibilities:, Assist i..."
...,...,...,...,...,...,...,...,...,...,...
79,Rwanda Tree Lead,one-acre-fund,,15-09-2022,20-11-2022,RW,1,Full-time,,"[About One Acre Fund, Founded in 2006, One Acr..."
80,Rwanda Potato Seed Venture ﻿﻿Lead,one-acre-fund,Senior (5+ years of experience),14-09-2022,28-11-2022,RW,1,Full-time,Master,"[About One Acre Fund, Founded in 2006, One Acr..."
81,IT Operations Senior Manager,one-acre-fund,Not specified,07-09-2022,06-12-2022,RW,1,Full-time,Not specified,"[About One Acre Fund, Founded in 2006, One Acr..."
82,Rwanda Seed Innovation Centre Lead,one-acre-fund,,01-09-2022,29-11-2022,RW,1,Full-time,,"[About One Acre Fund, Founded in 2006, One Acr..."


In [101]:
content =  requests.get('https://www.jobinrwanda.com/').content
soup__ =  BeautifulSoup(content, "html.parser")

In [103]:
link__ = 'https://www.jobinrwanda.com/' + soup__.find('a',class_='nav-link px-1 text-primary nav-link--jobs-all')['href']
print(link__)

https://www.jobinrwanda.com//jobs/all


In [105]:
linka = 'https://www.jobinrwanda.com/' + soup__.find('a',class_='nav-link px-1 text-primary nav-link--jobs-consultancy')['href']
print(linka)

https://www.jobinrwanda.com//jobs/consultancy
